In [1]:
import os
import sys
from pathlib import Path
from typing import List

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io
from skimage.measure import regionprops_table
from skimage import exposure, measure
from tqdm.notebook import trange, tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

from config import *

In [4]:
module_path = str(Path.cwd().parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

import utils as my_utils

In [5]:
# Define image and masks folder
info_csv = data_meta / "all_info.csv"

df = pd.read_csv(info_csv)

In [6]:
df_subset = df[(df.Location == 36) & (df.After_bleach == False)]

In [7]:
df_subset 

,Cycle,Location,After_bleach,Z_stack,Channel,Marker,Path,Path_corrected
2625,Cycle10,36,False,1,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
2626,Cycle10,36,False,1,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
2627,Cycle10,36,False,1,CH4,PCNA,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
2628,Cycle10,36,False,2,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
2629,Cycle10,36,False,2,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
...,...,...,...,...,...,...,...,...
84519,Cycle9,36,False,24,CH4,H3k4me3-647,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
84520,Cycle9,36,False,25,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
84521,Cycle9,36,False,25,CH2,H3k27me3-488,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
84522,Cycle9,36,False,25,CH3,H3k9Ac-555,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...


# Get images

In [40]:
from joblib import Parallel, delayed\

def read_img(path):
    return contrast_streching(skimage.io.imread(path))

def contrast_streching(img):
    p2, p98 = np.percentile(img, (0.5, 99.5))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def non_border_cell(cell_mask):
    labels = []
    row_max, col_max = cell_mask.shape
    props = measure.regionprops(cell_mask)
    for prop in props:
        if prop['bbox'][0] == 0 or prop['bbox'][1] == 0 or prop['bbox'][2] == row_max or prop['bbox'][3] == col_max:
            continue
        else:
            labels.append(prop['label'])
    return labels

def shift_crop(imgs, cycles, shift_dict):
    '''
    Shift and crop image from on region and on image
    '''
    max_shift_x = shift_dict["max"]["max_shift_x"]
    min_shift_x = shift_dict["max"]["min_shift_x"]
    max_shift_y = shift_dict["max"]["max_shift_y"]
    min_shift_y = shift_dict["max"]["min_shift_y"]
    imgs_shift_crop = []
    for idx, img in tqdm(enumerate(imgs), total=len(imgs), leave=False):
        
        cycle = cycles[idx]
        
        # If cycle 1 only crop else shift crop
        if cycle == 'Cycle1':
            res_cropped = img[max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
        else:
            # Aplly shift
            shift_x, shift_y = (
                shift_dict[cycle]['norm']["shift_x"],
                shift_dict[cycle]['norm']["shift_y"],
            )
            rows, cols = img.shape
            M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
            res = cv2.warpAffine(img, M, (cols, rows))

            # Crop image
            res_cropped = res[max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
        
        imgs_shift_crop.append(res_cropped)
    return imgs_shift_crop

In [41]:
z_stacks = df_subset.Z_stack.unique()

In [42]:
masks = my_utils.get_masks(data_mask)
masks = masks['36']

def apply_mask(img, mask=masks['cyto']):
    img = img[:mask.shape[0], :mask.shape[1]]
    return np.where(mask>0, img, 0)

In [43]:
import pickle 
from collections import defaultdict

shift_file = data_meta / 'shift.pkl'
with open(data_meta / 'shift.pkl', 'rb') as file:
    shift_dict = pickle.load(file)

In [66]:
imgs_dict = defaultdict(dict)

for z in tqdm(z_stacks, total=len(z_stacks)):
    df_z = df_subset[df_subset.Z_stack == z]
    cycles = df_z.Cycle.tolist()
    paths = df_z.Path_corrected.tolist()
    markers = df_z.Marker.tolist()

    imgs = joblib_loop(read_img, paths)
    imgs = shift_crop(imgs, cycles, shift_dict[36])
    imgs = joblib_loop(apply_mask, imgs)
    
    for i, img in enumerate(imgs):
        if imgs_dict[cycles[i]].get(markers[i]) == None:
            imgs_dict[cycles[i]][markers[i]] = [img]
        else:
            imgs_dict[cycles[i]][markers[i]].append(img)


  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
for cycle, markers in imgs_dict.items():
    for marker, imgs in markers.items():
        imgs = np.stack(imgs, axis=0)
        skimage.io.imsave(figure_dir / 'masked_image' / f'{cycle}_{marker}.tiff', imgs, plugin='tifffile')